In [25]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy import Column, Integer, String, Date, Float, create_engine, inspect, text

In [26]:
root_dir = os.getcwd()
db_dir = 'database'
db_name = 'dataset.db'
db_path = os.path.join(root_dir, db_dir, db_name)
db_url = f'sqlite:///{db_path}'

kaggle_data_set = 'eduardolicea/healthcare-dataset'

dataset_dir = 'dataset'
dataset_name = 'modified_healthcare_dataset.csv'
dataset_path = os.path.join(root_dir, dataset_dir, dataset_name)

separator = f"\n{'-'*100}\n"

In [27]:
Base = declarative_base()

class Gender(Base):
    __tablename__ = 'genders'
    id = Column(Integer, primary_key=True, autoincrement=True)
    Gender = Column(String)

class BloodType(Base):
    __tablename__ = 'blood_types'
    id = Column(Integer, primary_key=True, autoincrement=True)
    BloodType = Column(String)

class MedicalCondition(Base):
    __tablename__ = 'medical_conditions'
    id = Column(Integer, primary_key=True, autoincrement=True)
    MedicalCondition = Column(String)

class Doctor(Base):
    __tablename__ = 'doctors'
    id = Column(Integer, primary_key=True, autoincrement=True)
    Doctor = Column(String, unique=True)

class Hospital(Base):
    __tablename__ = 'hospitals'
    id = Column(Integer, primary_key=True, autoincrement=True)
    Hospital = Column(String)

class InsuranceProvider(Base):
    __tablename__ = 'insurance_providers'
    id = Column(Integer, primary_key=True, autoincrement=True)
    InsuranceProvider = Column(String)

class AdmissionType(Base):
    __tablename__ = 'admission_types'
    id = Column(Integer, primary_key=True, autoincrement=True)
    AdmissionType = Column(String)

class Medication(Base):
    __tablename__ = 'medications'
    id = Column(Integer, primary_key=True, autoincrement=True)
    Medication = Column(String)

class TestResults(Base):
    __tablename__ = 'test_results'
    id = Column(Integer, primary_key=True, autoincrement=True)
    TestResults = Column(String)


class Patient(Base):
    __tablename__ = 'patients'
    id = Column(Integer, primary_key=True)
    Name = Column(String)
    Age = Column(Integer)
    DateOfAdmission = Column(Date)
    BillingAmount = Column(Float)
    RoomNumber = Column(Integer)
    DischargeDate = Column(Date)
    LengthOfStay = Column(Integer)

class DataSet(Base):
    __tablename__ = 'dataset'
    id = Column(Integer, primary_key=True, autoincrement=True)
    Name = Column(String)
    Age = Column(Integer)
    Gender = Column(String)
    BloodType = Column(String)
    MedicalCondition = Column(String)
    DateOfAdmission = Column(Date)
    Doctor = Column(String)
    Hospital = Column(String)
    InsuranceProvider = Column(String)
    BillingAmount = Column(Float)
    RoomNumber = Column(Integer)
    AdmissionType = Column(String)
    DischargeDate = Column(Date)
    Medication = Column(String)
    TestResults = Column(String)
    LengthOfStay = Column(Integer)


In [28]:
def load_data_csv(file_path):
    """
    Загрузка данных из CSV файла.
    :param file_path: Путь к CSV файлу.
    :return: DataFrame с загруженными данными.
    """
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        raise Exception(e)


In [29]:
def load_kaggle_data_set(dataset):
    """
    Загрузка датасета из kaggle.

    Важно: Необходимо предварительно получить API Token.
    Переместите файл kaggle.json в папку ~/.kaggle/ (для Linux и macOS)
    или в C:/Users/<Ваше_имя_пользователя>/.kaggle/ (для Windows).
    Если папка .kaggle не существует, создайте ее.
    :param dataset:
    :return:
    """

    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files(dataset=dataset,
                               path='./dataset',
                               force=True,
                               unzip=True)              # Загрузка и распаковка

In [30]:
engine = create_engine(db_url, echo=False)
Session = sessionmaker(bind=engine)

In [31]:
def db_cerate():
    try:
        with Session()  as session:
            Base.metadata.create_all(engine)                # Создать базу данных (если она еще не существует)
            session.commit()
    except Exception as e:
        raise (f'Ошибка создания базы данных: {e}')

In [32]:
def add_all_from_dataset_to_table(df: pd.DataFrame,
                                  table_name : str,
                                  unique_columns : list = None):
    with Session() as session:
        truncate_table = text(f'DELETE FROM {table_name}')
        session.execute(truncate_table)
        session.commit()

    inspector = inspect(engine)

    columns = [col['name'] for col in inspector.get_columns(table_name)
               if col['name']  != 'id']                                                     # Получить список столбцов из таблицы 'patients', исключая id, т.к. он автоинкриментный в таблице
    if unique_columns:                                                                      # Если передан лист имен столбцов в которых значения д.б. уникальными
        df_filtered = df[columns].drop_duplicates(unique_columns)                           # Фильтр DataFrame, оставляя только те столбцы, которые есть в таблице, и дополнительный фильтр по уникальным
    else:
        df_filtered = df[columns]                                                           # Фильтр DataFrame, оставляя только те столбцы, которые есть в таблице

    df_filtered.to_sql(table_name, con=engine, if_exists='append', index=False)             # Запись DataFrame в таблицу

In [33]:
def fill_db_tabes(df: pd.DataFrame):
    add_all_from_dataset_to_table(df, 'genders', ['Gender'])
    add_all_from_dataset_to_table(df, 'blood_types', ['BloodType'])
    add_all_from_dataset_to_table(df, 'medical_conditions', ['MedicalCondition'])
    add_all_from_dataset_to_table(df, 'doctors', ['Doctor'])
    add_all_from_dataset_to_table(df, 'hospitals', ['Hospital'])
    add_all_from_dataset_to_table(df, 'insurance_providers', ['InsuranceProvider'])
    add_all_from_dataset_to_table(df, 'admission_types', ['AdmissionType'])
    add_all_from_dataset_to_table(df, 'medications', ['Medication'])
    add_all_from_dataset_to_table(df, 'test_results', ['TestResults'])
    add_all_from_dataset_to_table(df, 'patients')
    add_all_from_dataset_to_table(df, 'dataset')

In [34]:
# Создам представление
def create_view():
    drop_sql = """DROP VIEW IF EXISTS vw_patients"""
    create_sql = """
    CREATE VIEW vw_patients AS
    SELECT dataset.id,
            dataset.Name,
            dataset.Age,
            genders.id AS GenderID,
            blood_types.id AS BloodTypeID,
            medical_conditions.id AS MedicalConditionID,
            dataset.DateOfAdmission,
            doctors.id AS DoctorID,
            hospitals.id AS HospitalID,
            insurance_providers.id AS InsuranceProviderID,
            dataset.BillingAmount,
            dataset.RoomNumber,
            admission_types.id AS AdmissionTypeID,
            dataset.DischargeDate,
            medications.id AS MedicationID,
            test_results.id,
            dataset.LengthOfStay
    FROM dataset
    JOIN genders ON genders.Gender = dataset.Gender
    JOIN blood_types ON blood_types.BloodType = dataset.BloodType
    JOIN medical_conditions ON medical_conditions.MedicalCondition = dataset.MedicalCondition
    JOIN doctors ON doctors.Doctor = dataset.Doctor
    JOIN hospitals ON hospitals.Hospital = dataset.Hospital
    JOIN insurance_providers ON insurance_providers.InsuranceProvider = dataset.InsuranceProvider
    JOIN admission_types ON admission_types.AdmissionType = dataset.AdmissionType
    JOIN medications ON medications.Medication = dataset.Medication
    JOIN test_results ON test_results.TestResults = dataset.TestResults;
    """

    with engine.connect() as connection:
        try:
            connection.execute(text(drop_sql))
            connection.execute(text(create_sql))
        except Exception as e:
            raise (f'Ошибка создания представления: {e}')


In [35]:
    try:
        if not os.path.exists(dataset_path):
            load_kaggle_data_set(kaggle_data_set)               # Загрузка файла датасета из kaggle если он не загружен
        df = load_data_csv(dataset_path)                        # Загрузка файла в датафрейм

        df.info()                                           # Информация о датафрейме (датасете)

        print(separator,
              'Отсутствующие значения:\n',
              df.isna().sum())                              # Отсутствующие значения в столбцах датафрейма. Можно посмотреть и в информации, но так более наглядно.

        print(separator,
              'Количество дублирующихся значений: ',
              df.duplicated().sum())

        df.rename(columns={'Blood Type'         : 'BloodType',
                           'Medical Condition'  : 'MedicalCondition',
                           'Date of Admission'  : 'DateOfAdmission',
                           'Insurance Provider' : 'InsuranceProvider',
                           'Billing Amount'     : 'BillingAmount',
                           'Room Number'        : 'RoomNumber',
                           'Admission Type'     : 'AdmissionType',
                           'Discharge Date'     : 'DischargeDate',
                           'Test Results'       : 'TestResults',
                           'Length of Stay'     : 'LengthOfStay'
                           },
                  inplace=True
        )                                                                   # Переименовать столбцы лоя дальнейшей работы

        df.info()                                                           # Информация о датафрейме (датасете)
        db_cerate()                                                         # Создать БД, со всеми таблицами (таблицы пустые)
        fill_db_tabes(df)                                                   # Заполнить таблицы в БД соответствующими данными из датасета (датафрейма)
        create_view()

    except Exception as e:
        print(f'При выполнении произошла ошибка: {e}')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                55500 non-null  object 
 1   Age                 55500 non-null  int64  
 2   Gender              55500 non-null  object 
 3   Blood Type          55500 non-null  object 
 4   Medical Condition   55500 non-null  object 
 5   Date of Admission   55500 non-null  object 
 6   Doctor              55500 non-null  object 
 7   Hospital            55500 non-null  object 
 8   Insurance Provider  55500 non-null  object 
 9   Billing Amount      55500 non-null  float64
 10  Room Number         55500 non-null  int64  
 11  Admission Type      55500 non-null  object 
 12  Discharge Date      55500 non-null  object 
 13  Medication          55500 non-null  object 
 14  Test Results        55500 non-null  object 
 15  Length of Stay      55500 non-null  int64  
dtypes: f